# Deep Reinforcement Learning

Mehrafarin Kazemi: 99109488

Ali Shahheidar: 400105777

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [1]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [2]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq
!pip install opencv-python-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [4]:
env_name = 'CartPole-v1'
env = gym.make(env_name)
obs = env.reset()
frames = []

for _ in range(500):
    action = env.action_space.sample()

    res = env.step(action)
    obs = res[0]
    done = res[2]
    frame = env.render(mode='rgb_array')
    frame = cv2.resize(frame, (608, 400))
    frames.append(frame)

    if done:
        obs = env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

In [5]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [6]:
class ActorCritic(nn.Module):
    def __init__(self, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        self.input_size = 4
        self.hidden_layer = nn.Sequential(
            nn.Linear(self.input_size, hidden_size),
            nn.ReLU()
        )
        self.actor_layer = nn.Sequential(
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=-1)
        )
        self.critic_layer = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.hidden_layer(x)
        action_probs = self.actor_layer(x)
        value = self.critic_layer(x)
        return action_probs, value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [7]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = int(num_episodes)
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # define your actor critic network, its optimzer and the loss function
        self.policy_net = ActorCritic(hidden_size, self.env.action_space.n).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr = lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        # TODO
        # choose an action by sampling from the policy distribution
        action = torch.multinomial(self.policy_net(state)[0], 1).item()
        return action

    def compute_returns(self, rewards):
        # TODO
        # calculate the discounted rewards used in the training process
        discounted_reward = 0
        dst = [0] * len(rewards)

        for i in reversed(range(len(rewards))):
            discounted_reward *= self.gamma
            discounted_reward += rewards[i]
            dst[i] = discounted_reward

        dst = torch.tensor(dst).to(self.device)
        return dst

    def train(self):
        episode_rewards = np.array([])
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                # TODO
                # Implement the training loop
                state = self.env.reset()
                state = torch.FloatTensor(state).to(self.device)

                episode_reward = 0

                values = []
                rewards = []
                log_probabilities = []

                for step in range(self.max_steps):
                    # TODO
                    # first you need to gather log probabilities, state values and rewards from a trajectory
                    # update the 'episode_reward'
                    action_probs, value = self.policy_net(state)
                    values.append(value)

                    action = self.choose_action(state)
                    log_probabilities.append(torch.log(action_probs.squeeze(0)[action]))

                    res = self.env.step(action)
                    state = torch.FloatTensor(res[0]).to(self.device)

                    reward = res[1]
                    episode_reward += reward
                    rewards.append(reward)

                    if res[2]:
                        break

                episode_rewards = np.append(episode_rewards, episode_reward)

                # calculate the discounted rewards
                returns = self.compute_returns(rewards)

                # print("values", values)
                # print("returns", returns)
                values = torch.cat(values).squeeze(-1)
                # print(f"Episode {episode} - Values shape: {values.shape}, Returns shape: {returns.shape}")

                # calculate advantage
                advantage = returns - values

                # compute actor and critic losses
                actor_loss = -(advantage.detach() * torch.stack(log_probabilities)).mean()
                critic_loss = self.critic_loss(values, returns)
                loss = actor_loss + critic_loss

                # TODO
                # update the model after performing the backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                pbar.set_description(f"Episode {episode}, Reward: {np.mean(episode_reward)}")

        self.env.close()
        return episode_rewards

Define the model and set hyperparameters.

In [8]:
env = gym.make('CartPole-v1')
# num_episodes = 2200
# max_steps = 1000
# lr = 1e-3
# hidden_size = 512
# gamma = 0.99
# 17 min - average 480

# num_episodes = 1200
# max_steps = 1000
# lr = 1e-3
# hidden_size = 1024
# gamma = 0.99
# 8 min - average 490


# num_episodes = 1000
# max_steps = 2000
# lr = 1e-3
# hidden_size = 512
# gamma = 0.99
# average 341.0
# 12 min - average 440

num_episodes = 1500
max_steps = 1000
lr = 1e-3
hidden_size = 1024

a2c_model = A2CAgent(env, num_episodes, max_steps, lr=lr, hidden_size=hidden_size)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Train the model.

In [9]:
rewards = a2c_model.train()

  0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Episode 1499, Reward: 500.0: 100%|██████████| 1500/1500 [07:23<00:00,  3.38it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [10]:
env = gym.make(env_name, render_mode='rgb_array')

model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

frames = []
episode_rewards = []

def resize_frame(frame):
    height, width = frame.shape[:2]
    new_height = (height + 15) // 16 * 16
    new_width = (width + 15) // 16 * 16
    resized_frame = np.zeros((new_height, new_width, frame.shape[2]), dtype=frame.dtype)
    resized_frame[:height, :width] = frame
    return resized_frame

for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0

    while True:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            action = a2c_model.choose_action(state)

        res = env.step(action)
        state = res[0]
        episode_reward += res[1]

        if i == 0:
            frame = env.render()
            frame = np.array(frame)
            if len(frame.shape) == 4 and frame.shape[0] == 1:
                frame = frame[0]
            if len(frame.shape) == 3 and frame.shape[2] in [1, 2, 3, 4]:
                frame = resize_frame(frame)
                frames.append(frame)
        if res[2]:
            break

    episode_rewards.append(episode_reward)
    print(f"Episode {i} Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
imageio.mimsave('./test.mp4', frames, fps=25)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Episode 0 Reward: 500.0
Episode 1 Reward: 500.0
Episode 2 Reward: 500.0
Episode 3 Reward: 500.0
Episode 4 Reward: 500.0
Episode 5 Reward: 500.0
Episode 6 Reward: 500.0
Episode 7 Reward: 500.0
Episode 8 Reward: 500.0
Episode 9 Reward: 500.0
Average Reward over 10 episodes: 500.0


In [11]:
show_video('./test.mp4')